# Another TimeFunc method

I currently have a truncate method. I also want to have a stretch method. This scales the scale pattern up and down.

So if the pattern is `[3/4, 1 1/2, 2]`, then stretching it by a factor of 3/4 yields `[1/2, 1 1/8, 1 1/2]`.

In [19]:
from harmonica.time import Clip, TimeFunc
from harmonica.utility import GMDrum, Mixed

clip_len = Mixed(128)

f = TimeFunc([Mixed(x) for x in ["3/4", "1 1/2", 2]])
g = f.stretch(Mixed("3/4")).trunc(Mixed(4))
h = f.stretch(Mixed("1 1/2")).trunc(Mixed(4))
f = f.trunc(Mixed(4))
t = TimeFunc([Mixed(1)])
c = t.trunc(Mixed(4)).pad_tail(Mixed(4)).shift(Mixed(4))
k = TimeFunc(
    [Mixed(x) for x in ["1/3", "2/3", "1 1/3", "1 2/3", 2, 32]], offset=Mixed("18 1/2")
)
j = TimeFunc([Mixed(x) for x in ["1/4", "3/4", "2 1/2", "3 1/2", 4]])

Clip(
    [
        f.to_clip(clip_len, drum=GMDrum.HiWoodBlock),
        g.to_clip(clip_len, drum=GMDrum.Claves),
        h.to_clip(clip_len, drum=GMDrum.AcousticBassDrum),
        t.to_clip(clip_len, drum=GMDrum.PedalHiHat),
        c.to_clip(clip_len, drum=GMDrum.HandClap),
        k.to_clip(clip_len, drum=GMDrum.LowMidTom),
        j.to_clip(clip_len, drum=GMDrum.ShortGuiro),
    ]
).preview(tempo=100)

Wow, I got... carried away with that one. Lots of fun!

That's a good sign, though. That means I've created some musically interesting tools!

# The desire for musical L-systems

I have recently learned about L-systems. They seem to basically be similar to formal grammars, except in each iteration, all of the rewrite rules are applied in parallel.

I want to construct chord progressions with this _somehow_. But maybe not really. I just want some sort of iterative techniques for building out progressions.

One idea is to start with a single chord, and have it occupy some region of time. Say, an 8 bar loop, so 32 beats.

There's a limit to how fast you can change chords in a progression before it starts sounding like nonsense. This is subject and context-dependent.

It's also probably kind of boring to change chords at a constant pace. Harmonic rhythm should vary!



# Diatonic transposition of pitch class sets

Chromatic transposition of a pitch class set is trivial. But to perform diatonic transposition requires another object, which I call the index set.

The pitch class set must also have a root for this to work. 

You have a pitch class set like `{0,2,3,5,7,8,10} mod 12 root 0`, and then you have an index set like `{0,2,4} mod 7 root 0` representing a triad.

You select pitches from the pcset with this index set, and it yields `{0,3,7} mod 12 root 0`. Now, to do diatonic transposition, you transpose the index class set, not the pitch class set, and then you select with it.

The question is, how do you implement this?

Like, I could see there being something like

```
scale = PitchClassSet([0,2,3,5,7,8,10], modulus=12, root=3)
selector = DiatonicSet([0,2,4], modulus=7, root=0)
```

but how do you derive the actual diatonic chord, i.e. subset? 

`scale.diatonic(selector)` could yield `PitchClassSet([3,7,10], modulus=12, root=3)`
or the same could be yielded by `selector.eval(scale)`? Ehhh...

And maybe `diatonic` isn't the best name. I'm just selecting elements from a set, after all. Why not just call it `select`?

So `scale.select(selector)`. Yeah, seems right to me.

At this point, though, I'm starting to wonder if it even makes sense to name it something different. As in, why not just do away with the whole "DiatonicSet" thing? Just use PitchClassSets.

Of course, this feels a bit awkward, and like a semantic abuse of sorts. But hey, if it works...

In [1]:
from harmonica.pitch._scales import PitchClassSet


selector = PitchClassSet([0, 2, 4], modulus=7, root=0)
scale = PitchClassSet([0, 2, 3, 5, 7, 8, 10], modulus=12, root=3)

print(scale)
print(scale.select(selector))
print(scale.select(selector + 1))
print(scale.select(selector + 2))
print(scale.select(selector + 3))
print(scale.select(selector - 1))
print(scale.select(selector - 2))
print(scale.select(selector - 3))

PitchClassSet(pitch_classes=[0, 2, 3, 5, 7, 8, 10], modulus=12, root=3)
PitchClassSet(pitch_classes=[3, 7, 10], modulus=12, root=3)
PitchClassSet(pitch_classes=[0, 5, 8], modulus=12, root=5)
PitchClassSet(pitch_classes=[2, 7, 10], modulus=12, root=7)
PitchClassSet(pitch_classes=[0, 3, 8], modulus=12, root=8)
PitchClassSet(pitch_classes=[2, 5, 8], modulus=12, root=2)
PitchClassSet(pitch_classes=[0, 3, 7], modulus=12, root=0)
PitchClassSet(pitch_classes=[2, 5, 10], modulus=12, root=10)


Yeah, that works just fine. Alright, cool. 

And I guess if I ever have a chord that I want to move around diatonically, I can just look up pitch class sets it fits in, then derive the selector that selects that chord from the parent scale, and then transpose that selector around to derive diatonically related chords and such.

Very cool!